In [ ]:
def create_model_test(ob_space):   
    import numpy as np
    import tensorflow as tf

    class Normc_initializer(tf.keras.initializers.Initializer):
        """
        Create a TensorFlow constant with random numbers normed in the given shape.
        :param std:
        :return:
        """
        def __init__(self, std=1.0):
            self.std = std

        def __call__(self, shape, dtype=None, partition_info=None):
            out = np.random.randn(*shape).astype(np.float32)
            out *= self.std / np.sqrt(np.square(out).sum(axis=0, keepdims=True))
            return tf.constant(out)
        
    class ObservationNormalizationLayer(tf.keras.layers.Layer):
        def __init__(self, ob_mean, ob_std, **kwargs):
            self.ob_mean = ob_mean
            self.ob_std = ob_std
            super(ObservationNormalizationLayer, self).__init__(**kwargs)

        def call(self, input):
            return tf.clip_by_value((input - self.ob_mean) / self.ob_std, -5.0, 5.0)
          
        def get_config(self):
            base_config = super(ObservationNormalizationLayer, self).get_config()
            base_config['ob_mean'] = self.ob_mean
            base_config['ob_std'] = self.ob_std
            return base_config
        
        @classmethod
        def from_config(cls, config):
            return cls(**config)
    
    input_layer = x = tf.keras.Input(ob_space.shape, dtype=tf.float32)
    
    a = ObservationNormalizationLayer(np.ones(ob_space.shape), np.ones(ob_space.shape))(x)
    
    model = tf.keras.Model(inputs=input_layer, outputs=a, name="test")

    return model

In [ ]:
import gym, roboschool
import numpy as np
env = gym.make("RoboschoolAnt-v1")

test_model = create_model_test(env.observation_space)
test_model.summary()

a = np.ones(shape=(1, 28)) * 4.5

print(a)

test_model.predict(a)